# 1. Agafar el text d'una web

In [38]:
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# # Descargar recursos necesarios de NLTK
nltk.download('punkt_tab')
nltk.download('gutenberg')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Alvaro\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\Alvaro\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Alvaro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [39]:
def title_article_preprocessing_a(clean_text):
    # 1. Tokenizar el texto
    tokens = word_tokenize(clean_text, language='spanish')
    
    # 2. Normalizar: pasar a minúsculas y quitar puntuación
    tokens = [t.lower() for t in tokens if t.isalpha()]  # solo palabras
    
    # 3. Eliminar palabras vacías (stopwords)
    stop_words = set(stopwords.words('spanish'))
    tokens = [t for t in tokens if t not in stop_words]
    
    print(tokens[:50])  # mostrar los primeros 50 tokens procesados


def title_article_preprocessing_b(clean_text):
    pass

In [40]:
import requests
from bs4 import BeautifulSoup

url = "https://link.springer.com/article/10.1186/s12915-020-00925-x"

opcio = input("Selecciona opció A o B")


def scrape_text(url, opcio):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    # 1. Intentar <article>
    article = soup.find("article")
    if article:
        text = article.get_text(separator="\n")
    else:
        # 2. Intentar <main>
        main = soup.find("main")
        if main:
            text = main.get_text(separator="\n")
        else:
            # 3. Fallback: todos los <p>, unir el bloque más largo
            paragraphs = soup.find_all("p")
            blocks = []
            current_block = []
            for p in paragraphs:
                current_block.append(p.get_text())
                # Separar bloques grandes por lógica simple
                if len(current_block) > 3:  
                    blocks.append("\n".join(current_block))
                    current_block = []
            if current_block:
                blocks.append("\n".join(current_block))
            # Elegir el bloque más largo (probable artículo)
            text = max(blocks, key=len) if blocks else ""
    
    # Limpiar líneas vacías
    lines = [line.strip() for line in text.splitlines() if line.strip()]
    clean_text = "\n".join(lines)
    

    print(clean_text[:1000])  # Mostrar solo los primeros 1000 caracteres
    if opcio == "A":
        title_article_preprocessing_a(clean_text)
    elif opcio == "B":
        title_article_preprocessing_b(clean_text)
    else:
        print("Opció no vàlida")


scrape_text(url, opcio)


Home
BMC Biology
Article
Comparative analysis of morabine grasshopper genomes reveals highly abundant transposable elements and rapidly proliferating satellite DNA repeats
Research article
Open access
Published:
21 December 2020
Volume 18
, article number
199
, (
2020
)
Cite this article
You have full access to this
open access
article
Download PDF
Save article
View saved research
BMC Biology
Aims and scope
Submit manuscript
Comparative analysis of morabine grasshopper genomes reveals highly abundant transposable elements and rapidly proliferating satellite DNA repeats
Download PDF
Octavio M. Palacios-Gimenez
ORCID:
orcid.org/0000-0002-1472-9949
1
,
2
,
Julia Koelman
1
,
Marc Palmada-Flores
1
,
Tessa M. Bradford
3
,
4
,
Karl K. Jones
3
,
Steven J. B. Cooper
3
,
4
,
Takeshi Kawakami
1
,
5
na1
&
…
Alexander Suh
1
,
2
,
6
na1
Show authors
6436
Accesses
47
Citations
12
Altmetric
Explore all metrics
Abstract
Background
Repetitive DNA sequences, including transposable elements (TEs) and tand